In [1]:
import pandas as pd
from datetime import datetime as dt
from matplotlib import pyplot as plt
from binance.client import Client
from binance.exceptions import BinanceAPIException
from misc import *
from secrets import secrets

In [17]:
BINANCE_API_KEY = secrets['BINANCE_API_KEY']
BINANCE_SECRET_KEY = secrets['BINANCE_SECRET_KEY']
BRIDGE = 'USDT'

client = Client(BINANCE_API_KEY, BINANCE_SECRET_KEY)

In [14]:
def stimulate_trade_bot1(client, coin, limit=0.01, display=True):
    symbol = coin + BRIDGE
    kline = client.get_klines(symbol=symbol, interval=Client.KLINE_INTERVAL_30MINUTE)
    # Initial
    initial = 1000
    usdt = 1000
    coin = 0
    bought = False
    last_buy_price = 0
    last_sell_price = 0
    trans_fee = 0.001
    # Split data
    train, test = kline[:400], kline[-100:]
    # Stimmy
    for t in test:
        data = train + [t]
        k, d, j = get_kdj1(data)
        if not bought:
            if d - k < limit:
                buy_price = (float(t[1]) + float(t[4])) / 2
                coin = (usdt / buy_price) * (1 - trans_fee)
                usdt = 0
                bought = True
                if display:
                    print('Bought {} @ {}. Current networth: {}'.format(coin, buy_price, coin * buy_price))
        else:
            if d - k > limit:
                sell_price = (float(t[1]) + float(t[4])) / 2
                usdt = (coin * sell_price) * (1 - trans_fee)
                bnb = 0
                bought = False
                if display:
                    print('Sold {} @ {}. Current networth: {}'.format(coin, sell_price, usdt))
    # Summary
    if usdt == 0:
        usdt = coin * buy_price
    return usdt, (usdt - initial) / initial * 100

def stimulate_trade_bot2(client, coin, limit, k_periods, d_periods):
    symbol = coin + BRIDGE
    kline = client.get_klines(symbol=symbol, interval=Client.KLINE_INTERVAL_30MINUTE)
    # Initial
    initial = 1000
    usdt = 1000
    coin = 0
    bought = False
    last_buy_price = 0
    last_sell_price = 0
    trans_fee = 0.001
    # Split data
    train, test = kline[:400], kline[-100:]
    # Stimmy
    for t in test:
        data = train + [t]
        k_value, d_value, j_value = get_kdj2(data, k_periods=k_periods, d_periods=d_periods)
        k = k_value[-1]
        d = d_value[-1]
        j = j_value[-1]
        if not bought:
            if d - k < limit:
                buy_price = (float(t[1]) + float(t[4])) / 2
                coin = (usdt / buy_price) * (1 - trans_fee)
                usdt = 0
                bought = True
                # print('Bought {} @ {}. Current networth: {}'.format(coin, buy_price, coin * buy_price))
        else:
            if d - k > limit:
                sell_price = (float(t[1]) + float(t[4])) / 2
                usdt = (coin * sell_price) * (1 - trans_fee)
                bnb = 0
                bought = False
                # print('Sold {} @ {}. Current networth: {}'.format(coin, sell_price, usdt))
    # Summary
    if usdt == 0:
        usdt = coin * buy_price
    return usdt, (usdt - initial) / initial * 100

In [19]:
stimulate_trade_bot1(client, 'BTC')

Bought 0.019898751408125484 @ 50204.155. Current networth: 999.0
Sold 0.019898751408125484 @ 50863.35. Current networth: 1011.1050402770448
Bought 0.019932361288338953 @ 50676.08. Current networth: 1010.0939352367678
Sold 0.019932361288338953 @ 50551.17. Current networth: 1006.5965798042531


(1006.5965798042531, 0.659657980425311)

In [20]:
stimulate_trade_bot1(client, 'ETH')

Bought 0.6320784561847517 @ 1580.5. Current networth: 999.0
Sold 0.6320784561847517 @ 1578.28. Current networth: 996.5991890414426
Bought 0.630577208361904 @ 1578.875. Current networth: 995.6025898524011
Sold 0.630577208361904 @ 1635.96. Current networth: 1030.5674907019486
Bought 0.6365224819537335 @ 1617.44. Current networth: 1029.5369232112469
Sold 0.6365224819537335 @ 1601.94. Current networth: 1018.6511539162229
Bought 0.6398313094634333 @ 1590.47. Current networth: 1017.6325027623067
Sold 0.6398313094634333 @ 1606.135. Current networth: 1026.6278047648264
Bought 0.6412452103364792 @ 1599.3899999999999. Current networth: 1025.6011769600614
Sold 0.6412452103364792 @ 1599.1100000000001. Current networth: 1024.3962066728661
Bought 0.6463883998485318 @ 1583.2150000000001. Current networth: 1023.3718104661933
Sold 0.6463883998485318 @ 1591.58. Current networth: 1027.7500705814953
Bought 0.6451023492668624 @ 1591.565. Current networth: 1026.722320510914
Sold 0.6451023492668624 @ 1583.26

(1030.5480977571765, 3.0548097757176493)

In [21]:
stimulate_trade_bot1(client, 'BNB')

(1000, 0.0)

In [22]:
stimulate_trade_bot1(client, 'ADA')

Bought 821.2016341830318 @ 1.21651. Current networth: 999.0
Sold 821.2016341830318 @ 1.223945. Current networth: 1004.1005285160008
Bought 817.1665278954362 @ 1.22753. Current networth: 1003.0964279874847
Sold 817.1665278954362 @ 1.23079. Current networth: 1004.7546304775555
Bought 816.8470925912693 @ 1.2288100000000002. Current networth: 1003.7498758470778
Sold 816.8470925912693 @ 1.22737. Current networth: 1001.5710424177125
Bought 813.1174419055897 @ 1.2305350000000002. Current networth: 1000.5694713752949
Sold 813.1174419055897 @ 1.227565. Current networth: 997.1563580602624
Bought 813.6793926986414 @ 1.224265. Current networth: 996.1592017022022
Sold 813.6793926986414 @ 1.2287. Current networth: 998.7681019390119
Bought 814.4789834103972 @ 1.22504. Current networth: 997.7693338370728
Sold 814.4789834103972 @ 1.23475. Current networth: 1004.6722468412219
Bought 818.1116673277693 @ 1.22681. Current networth: 1003.6675745943807
Sold 818.1116673277693 @ 1.22949. Current networth: 1004

(993.671141078613, -0.6328858921386995)

In [23]:
stimulate_trade_bot1(client, 'BAT')

Bought 1752.1704814522493 @ 0.57015. Current networth: 999.0
Sold 1752.1704814522493 @ 0.58075. Current networth: 1016.5554340962905
Bought 1744.4625588975252 @ 0.58215. Current networth: 1015.5388786621942
Sold 1744.4625588975252 @ 0.6034999999999999. Current networth: 1051.7303711403615
Bought 1789.9125055693717 @ 0.587. Current networth: 1050.6786407692211
Sold 1789.9125055693717 @ 0.57955. Current networth: 1036.3064488101265
Bought 1790.1956464833413 @ 0.5783. Current networth: 1035.2701423613164
Sold 1790.1956464833413 @ 0.5775. Current networth: 1032.8041478582854
Bought 1778.3029019483404 @ 0.5802. Current networth: 1031.7713437104271


(1031.7713437104271, 3.1771343710427122)

In [10]:
stimulate_trade_bot1(client, 'FTM')

Bought 1784.7412661122478 @ 0.5597449999999999. Current networth: 999.0
Sold 1784.7412661122478 @ 0.56409. Current networth: 1005.7479461004566


(1005.7479461004566, 0.5747946100456601)

In [18]:
coins = ['ADA', 'BNB', 'BAT', 'FTM']
limits = [0, 0.01, 0.05, 0.1, 0.5]
dict_list = []

for limit in limits:
    initial = 1000 * len(coins)
    total_net = 0
    for coin in coins:
        net, growth = stimulate_trade_bot1(client, coin, limit, display=False)
        total_net += net
    dict_list.append({'limit': limit, 'total': total_net, 'growth': total_net / initial * 100})

df = pd.DataFrame.from_dict(dict_list)
df = df.sort_values(by='growth', ascending=False)
df

,limit,total,growth
4,0.50,4021.910686,100.547767
3,0.10,4015.820840,100.395521
0,0.00,4013.170512,100.329263
1,0.01,4013.170512,100.329263
2,0.05,4013.170512,100.329263


In [ ]:
kdj_pairs = [(5, 3), (9, 3), (14, 3), (9, 5), (14, 5), (21, 5), (21, 7), (21, 14)]
coins = ['ADA', 'BNB', 'BAT', 'FTM']
limits = [0, 0.01, 0.05, 0.1, 0.5]
dict_list = []

for p in kdj_pairs:
    for limit in limits:
        initial = 1000 * len(coins)
        total_net = 0
        for coin in coins:
            net, growth = stimulate_trade_bot2(client, coin, limit, p[0], p[1])
            total_net += net
        dict_list.append({'pair': p, 'limit': limit, 'growth': total_net / initial * 100})

In [ ]:
df = pd.DataFrame.from_dict(dict_list)
df = df.sort_values(by='growth', ascending=False)
df.head()

In [ ]:
df